<a href="https://colab.research.google.com/github/desmond-rn/projet-inverse-2d/blob/master/src/notebook/format_binaire.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [57]:
%reset -f

In [58]:
import numpy as np
import random

In [59]:
np.set_printoptions(precision = 3)

# Traitement des données

## Chargement des données

""" DEPUIS GOOGLE DRIVE """



In [60]:
# from google.colab import drive
# drive.mount('/gdrive')

In [61]:
# %cd ~
# %cd /gdrive/My\ Drive
# %cd Colab\ Notebooks/Projet\ inverse/data
# print()

# %ls

""" EN LOCAL """

In [62]:
%cd ../data/forwardPB/outputs
# %ls

[WinError 3] The system cannot find the path specified: '../data/forwardPB/outputs'
c:\Users\Roussel\OneDrive\Unistra\SEMESTRE 3\Projets\MOCO\RECOVERY\data\forwardPB\outputs


Les donnees sont sauvegardées dans un format binaire intitulé **SDS** (source-densité-signal). La version 02 de ce format est structurée dans l'ordre ci-après (l'en-tete du fichier est compris entre les lignes horizontales, et le reste represente la structure d'une des 4 simulations presente dans le fichier):

---
- **`5`** octets pour la constante magique **sds02**
- __`2`__ octets pour le nombre de simulations presentes dans le fichier (4 simus)
- __`2`__ octets pour `N` (nombre de mailles en horizontale)
- __`2`__ octets pour `M` (nombre de mailles en verticale)
- __`2`__ octets pour `step_count` (nombre d'iterations en temps)
- **`1`** octet pour le charactere 0xA (newline)
---
- **`1`** octet pour le bord sur lequel se trouve la source: (0 pour up, 1 pour down, 2 pour left, 3 pour right, et -1 si aucun de ces 4)
- **`1`** octet pour la position de la source sur son bord: (-1 indiquant que la source couvre tout le bord)
- __`M * N * 4`__ octets pour la densite (multiplication par 4 car `float`)
- **`step_count * N * 4`** octets pour E_up
- idem pour F_up
- idem pour T_up
- **`step_count * N * 4`** octets pour E_down
- idem pour F_up
- idem pour T_up
- **`step_count * M * 4`** octets pour E_left
- idem pour F_left
- idem pour T_left
- **`step_count * M * 4`** octets pour E_right
- idem pour F_right
- idem pour T_right
- **`1`** octet pour le charactere `0xA` (newline)

In [63]:
def read_sds_version02(file_path):

  file = open(file_path, "rb")

  sds_version = file.read(5)
  # print(sds_version)
  assert(sds_version == b'sds02')

  simu_count = int.from_bytes(file.read(2), byteorder='little', signed=False)
  N = int.from_bytes(file.read(2), byteorder='little', signed=False)
  M = int.from_bytes(file.read(2), byteorder='little', signed=False)
  step_count = int.from_bytes(file.read(2), byteorder='little', signed=False)

  assert(file.read(1) == b'\n')

  X = np.zeros(shape=(step_count, N, 4, 4, 3), dtype=np.float32)
  Y = np.zeros(shape=(M,N), dtype=np.float32)

  # print(step_count)

  while True:
    byte = file.read(1)
    if len(byte) < 1:   ## EOF
      break

    source_edge = int.from_bytes(byte, byteorder='little', signed=True)
    source_pos = int.from_bytes(file.read(1), byteorder='little', signed=True)
    assert (source_edge != -1 and source_pos == -1)
    # print("source ", source_edge, source_pos)

    rho_attr = file.read(M*N*4)
    Y[:,:] = np.frombuffer(rho_attr, dtype=np.float32).reshape(M,N)

    for e_signal in range(4): ## up, down, left, right pour le signal
      for ch in range(3): ## E, F, T
        signal = file.read(step_count*N*4)
        X[:, :, source_edge, e_signal, ch] = np.frombuffer(signal, dtype=np.float32).reshape(step_count, N)

    assert(file.read(1) == b'\n')

  return X, Y

In [66]:
X = np.empty(shape=(955,383, 64, 4, 4, 3))
Y = np.empty(shape=(955,64, 64, 1))

cpt=0
for i in range(1928,2892):
    if i<=2168 or i>=2178:

        filename = str(i)+".sds"
        # print("Working on", filename)
        X_i, Y_i = read_sds_version02(filename)

        X[cpt] = X_i
        Y[cpt] = Y_i[:, :, np.newaxis]
        
        cpt += 1

print(X.shape, Y.shape)

Working on 1928.sds
Working on 1929.sds
Working on 1930.sds
Working on 1931.sds
Working on 1932.sds
Working on 1933.sds
Working on 1934.sds
Working on 1935.sds
Working on 1936.sds
Working on 1937.sds
Working on 1938.sds
Working on 1939.sds
Working on 1940.sds
Working on 1941.sds
Working on 1942.sds
Working on 1943.sds
Working on 1944.sds
Working on 1945.sds
Working on 1946.sds
Working on 1947.sds
Working on 1948.sds
Working on 1949.sds
Working on 1950.sds
Working on 1951.sds
Working on 1952.sds
Working on 1953.sds
Working on 1954.sds
Working on 1955.sds
Working on 1956.sds
Working on 1957.sds
Working on 1958.sds
Working on 1959.sds
Working on 1960.sds
Working on 1961.sds
Working on 1962.sds
Working on 1963.sds
Working on 1964.sds
Working on 1965.sds
Working on 1966.sds
Working on 1967.sds
Working on 1968.sds
Working on 1969.sds
Working on 1970.sds
Working on 1971.sds
Working on 1972.sds
Working on 1973.sds
Working on 1974.sds
Working on 1975.sds
Working on 1976.sds
Working on 1977.sds


In [67]:
# arr = np.arange(0,383,383/64)
# cpt=0

# for i in range(0,1928):
#     if i<482 or i>=964:
#         filename = str(i)+".sds"
#         X_i, Y_i = read_sds_version02(filename)
#         print("Working on", filename)

#         Xprime = np.zeros(shape=(64, 64, 4, 4, 3))
#         for i in range(64):
#             Xprime[i] = X_i[int(arr[i])]

#         X_i = Xprime.reshape(64,64,-1)
#         Y_i = Y_i[:, :, np.newaxis]

#         np.save("../../inversePB/inputs/"+str(cpt)+".npy", X_i)
#         np.save("../../inversePB/outputs/"+str(cpt)+".npy", Y_i)

#         cpt +=1

In [68]:
## Resampling

widthX = X.shape[1]
bs = X.shape[0]

Xprime = np.zeros(shape=(bs,64, 64, 4, 4, 3))

arr = np.arange(0,widthX,widthX/64)

for i in range(64):
    Xprime[:,i] = X[:,int(arr[i])]

X = Xprime.reshape(bs,64,64,-1)
print(np.max(Xprime, axis=None))

11.791653633117676


In [69]:
## Randomisation

bs = X.shape[0]
arr = list(range(bs))
random.shuffle(arr)
# print(arr)

Xprime = np.zeros(shape=(bs,64, 64, 48))
Yprime = np.zeros(shape=(bs,64, 64, 1))

for i in range(bs):
    Xprime[i] = X[arr[i]]
    Yprime[i] = Y[arr[i]]

X = Xprime
Y = Yprime

In [70]:
## Normalisation

X = X / np.max(X, axis=None)
Y = Y / np.max(Y, axis=None)

## New axis on Y to indicate it only has 1 chennel out
# Y = Y[:,:,:,np.newaxis]

print(X.shape, Y.shape)

(955, 64, 64, 48) (955, 64, 64, 1)


In [71]:
lenTrain=600
lenVal=250

X_train, Y_train = X[:lenTrain], Y[:lenTrain]
print("train shapes:", X_train.shape, Y_train.shape)

X_val, Y_val = X[lenTrain:lenVal+lenTrain], Y[lenTrain:lenVal+lenTrain]
print("val shapes:", X_val.shape, Y_val.shape)

X_test, Y_test = X[lenTrain+lenVal:], Y[lenTrain+lenVal:]
print("test shapes:", X_test.shape, Y_test.shape)

train shapes: (600, 64, 64, 48) (600, 64, 64, 1)
val shapes: (250, 64, 64, 48) (250, 64, 64, 1)
test shapes: (105, 64, 64, 48) (105, 64, 64, 1)


In [72]:
np.save("../../inversePB/train/X2.npy", X_train)
np.save("../../inversePB/train/Y2.npy", Y_train)

np.save("../../inversePB/val/X2.npy", X_val)
np.save("../../inversePB/val/Y2.npy", Y_val)

np.save("../../inversePB/test/X2.npy", X_test)
np.save("../../inversePB/test/Y2.npy", Y_test)

Rajout des donnees

In [73]:
X_train = np.concatenate([X_train, np.load("../../inversePB/train/X1.npy")])
print(X_train.shape)
Y_train = np.concatenate([Y_train, np.load("../../inversePB/train/Y1.npy")])
print(Y_train.shape)
np.save("../../inversePB/train/X.npy", X_train)
np.save("../../inversePB/train/Y.npy", Y_train)

(1950, 64, 64, 48)
(1950, 64, 64, 1)


In [74]:
X_val = np.concatenate([X_val, np.load("../../inversePB/val/X1.npy")])
print(X_val.shape)
Y_val = np.concatenate([Y_val, np.load("../../inversePB/val/Y1.npy")])
print(Y_val.shape)
np.save("../../inversePB/val/X.npy", X_val)
np.save("../../inversePB/val/Y.npy", Y_val)

(750, 64, 64, 48)
(750, 64, 64, 1)


In [75]:
X_test = np.concatenate([np.load("../../inversePB/test/X1.npy"), X_test])
print(X_test.shape)
Y_test = np.concatenate([np.load("../../inversePB/test/Y1.npy"), Y_test])
print(Y_test.shape)
np.save("../../inversePB/test/X.npy", X_test)
np.save("../../inversePB/test/Y.npy", Y_test)

(183, 64, 64, 48)
(183, 64, 64, 1)
